This notebook can be used to calculate NA Corrected intensities as well as fractional enrichment for LCMS data with ppm varying with molecular mass. The example shows a dataset with C13 and N15 label:

 - auto_detect_indistinguish_dual_label.csv - demo raw MS intensity file containing intensities for C10H17N3O6S simulated using combinatorics by considering C13 indistinguishable with O17 and N15 indistinguishable with S34

In [1]:
import pandas as pd
import numpy as np
import re

from corna.inputs import maven_parser as parser
import corna.constants as const
from corna.helpers import replace_negatives_in_column, merge_multiple_dfs
from corna.algorithms.nacorr_lcms import na_correction
from corna.postprocess import fractional_enrichment


Reading raw file and merging with sample metadata if present, in this example running without sample metadata

In [2]:
raw_df = pd.read_csv('auto_detect_indistinguish_dual_label.csv')
sample_metadata = pd.DataFrame()

merged_df, iso_tracer_data, element_list = parser.read_maven_file(raw_df, sample_metadata)
merged_df.head()

,Name,Label,Formula,Sample,Intensity,Unlabeled Fragment
0,Glutathione,C12 PARENT,C10H17N3O6S,Sample 1,0.000,Glutathione
1,Glutathione,C13-label-1,C10H17N3O6S,Sample 1,0.000,Glutathione
2,Glutathione,C13-label-2,C10H17N3O6S,Sample 1,0.000,Glutathione
3,Glutathione,C13-label-3,C10H17N3O6S,Sample 1,0.178,Glutathione
4,Glutathione,C13-label-4,C10H17N3O6S,Sample 1,0.013,Glutathione


Dictionary containing natural abundance values for the common isotopes found in nature. It can be defined by the user or one can use the default values from the package. The format of the dictionary is as shown below: 

{E:[M0, M1, ..Mn]} where E is the element symbol and the natural abundance fraction is in the increasing order of masses. For example:

In [3]:
#user defined
na_dict={'C':[0.9889,0.0111],
           'H':[0.99985, 0.00015],
           'N':[0.9964,0.0036],
           'O':[0.9976,0.0004,0.002],
           'S':[0.950,0.0076,0.0424]}

Performing na_correction and using the dictionary above for NA values. For Orbitrap, for molecular mass of 307 (which is our input compound) and 293808 resolution, ppm is ~7, according to the formula from Su, Xiaoyang et al.,2017

\begin{equation*}
\frac{Δm}{m} = 1.66 × \frac{m^\frac{1}{2}}{(MinimalNominalResolution×√200)} × 10^6
\end{equation*}

which is our ppm_user_input 

In [4]:
na_corr_df, ele_corr_dict = na_correction(merged_df, iso_tracers=['C13', 'N15'], eleme_corr={}, na_dict=na_dict, autodetect=True, 
                                          res=293808,res_mw=307, instrument='orbitrap')
na_corr_df = replace_negatives_in_column(na_corr_df, const.NA_CORRECTED_WITH_ZERO, const.NA_CORRECTED_COL)
na_corr_df


The ppm requirement is at the boderline for C10H17N3O6S:S34


,Name,Formula,Sample,NA Corrected,Intensity,Label,NA Corrected with zero
0,Glutathione,C10H17N3O6S,Sample 1,-1.996475e-16,0.0000,C12 PARENT,0.000000e+00
1,Glutathione,C10H17N3O6S,Sample 1,-3.338077e-16,0.0000,N15-label-1,0.000000e+00
2,Glutathione,C10H17N3O6S,Sample 1,-1.376422e-16,0.0000,N15-label-2,0.000000e+00
3,Glutathione,C10H17N3O6S,Sample 1,8.725420e-01,0.6808,N15-label-3,8.725420e-01
4,Glutathione,C10H17N3O6S,Sample 1,2.163494e-17,0.0000,C13-label-1,2.163494e-17
5,Glutathione,C10H17N3O6S,Sample 1,-9.755989e-19,0.0000,C13N15-label-1-1,0.000000e+00
6,Glutathione,C10H17N3O6S,Sample 1,-2.612839e-19,0.0000,C13N15-label-1-2,0.000000e+00
7,Glutathione,C10H17N3O6S,Sample 1,2.639991e-03,0.0785,C13N15-label-1-3,2.639991e-03
8,Glutathione,C10H17N3O6S,Sample 1,1.676099e-16,0.0000,C13-label-2,1.676099e-16
9,Glutathione,C10H17N3O6S,Sample 1,1.385446e-17,0.0000,C13N15-label-2-1,1.385446e-17


Calculating fractional enrichments, merging all data into a single file and saving as 'auto_detect_dual_label_isotope_ppm7.csv'

In [5]:
frac_enr_df = fractional_enrichment(na_corr_df)
frac_enr_df

,Sample,Name,Label,Formula,Pool_total,Fractional enrichment
0,Sample 1,Glutathione,C12 PARENT,C10H17N3O6S,1.102166,0.000000e+00
1,Sample 1,Glutathione,N15-label-1,C10H17N3O6S,1.102166,0.000000e+00
2,Sample 1,Glutathione,N15-label-2,C10H17N3O6S,1.102166,0.000000e+00
3,Sample 1,Glutathione,N15-label-3,C10H17N3O6S,1.102166,7.916613e-01
4,Sample 1,Glutathione,C13-label-1,C10H17N3O6S,1.102166,1.962948e-17
5,Sample 1,Glutathione,C13N15-label-1-1,C10H17N3O6S,1.102166,0.000000e+00
6,Sample 1,Glutathione,C13N15-label-1-2,C10H17N3O6S,1.102166,0.000000e+00
7,Sample 1,Glutathione,C13N15-label-1-3,C10H17N3O6S,1.102166,2.395276e-03
8,Sample 1,Glutathione,C13-label-2,C10H17N3O6S,1.102166,1.520732e-16
9,Sample 1,Glutathione,C13N15-label-2-1,C10H17N3O6S,1.102166,1.257021e-17


In [6]:
output_df = merge_multiple_dfs([merged_df, na_corr_df, frac_enr_df])
output_df

,Name,Label,Formula,Sample,Intensity_x,Unlabeled Fragment,NA Corrected,Intensity_y,NA Corrected with zero,Pool_total,Fractional enrichment
0,Glutathione,C12 PARENT,C10H17N3O6S,Sample 1,0.0000,Glutathione,-1.996475e-16,0.0000,0.000000e+00,1.102166,0.000000e+00
1,Glutathione,C13-label-1,C10H17N3O6S,Sample 1,0.0000,Glutathione,2.163494e-17,0.0000,2.163494e-17,1.102166,1.962948e-17
2,Glutathione,C13-label-2,C10H17N3O6S,Sample 1,0.0000,Glutathione,1.676099e-16,0.0000,1.676099e-16,1.102166,1.520732e-16
3,Glutathione,C13-label-3,C10H17N3O6S,Sample 1,0.1780,Glutathione,2.230195e-01,0.1780,2.230195e-01,1.102166,2.023466e-01
4,Glutathione,C13-label-4,C10H17N3O6S,Sample 1,0.0130,Glutathione,-1.221470e-03,0.0130,0.000000e+00,1.102166,0.000000e+00
5,Glutathione,C13-label-5,C10H17N3O6S,Sample 1,0.0002,Glutathione,-9.984606e-03,0.0002,0.000000e+00,1.102166,0.000000e+00
6,Glutathione,C13-label-6,C10H17N3O6S,Sample 1,0.0001,Glutathione,-3.612291e-05,0.0001,0.000000e+00,1.102166,0.000000e+00
7,Glutathione,C13N15-label-1-3,C10H17N3O6S,Sample 1,0.0785,Glutathione,2.639991e-03,0.0785,2.639991e-03,1.102166,2.395276e-03
8,Glutathione,C13N15-label-1-1,C10H17N3O6S,Sample 1,0.0000,Glutathione,-9.755989e-19,0.0000,0.000000e+00,1.102166,0.000000e+00
9,Glutathione,C13N15-label-1-2,C10H17N3O6S,Sample 1,0.0000,Glutathione,-2.612839e-19,0.0000,0.000000e+00,1.102166,0.000000e+00


In [7]:
output_df.to_csv('auto_detect_dual_label_isotope_res293808_at307_orbitrap.csv')